In [1]:
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite
import sys, os
sys.path.append("../")
from bqm_input_checker import InputChecker
from btor2bqm import BTor2BQM

# 32 bit

In [2]:
parser = BTor2BQM(10)
bqm_32, total_time_32, num_variables_32 = parser.parse_file(f"../../../qa_examples/32_d.btor2",
                          f"./32_d/",
                          with_init=True, modify_memory_sort=True)
print(total_time_32, num_variables_32)

started building ../../../qa_examples/32_d.btor2 for 10 timesteps
{'size_datasegment': 1, 'size_heap': 3, 'size_stack': 20, 'begin_datasegment': 17408, 'begin_heap': 18432, 'word_size': 32, 'address_step_size': 1, 'address_word_size': 30, 'begin_stack': 1073741824}
output dir:  ./32_d/
sort memory modified to be bitvector of size:  768
0.17 1414


# 64 bit

In [3]:
parser = BTor2BQM(10)
bqm_64, total_time_64, num_variables_64 = parser.parse_file(f"../../../qa_examples/64_d.btor2",
                          f"./64_d/",
                          with_init=True, modify_memory_sort=True)
print(total_time_64, num_variables_64)

started building ../../../qa_examples/64_d.btor2 for 10 timesteps
{'size_datasegment': 1, 'size_heap': 2, 'size_stack': 15, 'begin_datasegment': 8704, 'begin_heap': 9216, 'word_size': 64, 'address_step_size': 1, 'address_word_size': 29, 'begin_stack': 536870912}
output dir:  ./64_d/
sort memory modified to be bitvector of size:  1152
0.36 2790


# Minor Embedding

In [4]:
import minorminer
import dimod
import dwave_networkx as dnx
problem_graph_32 = dimod.to_networkx_graph(bqm_32)
problem_graph_64 = dimod.to_networkx_graph(bqm_64)
chimera_graph = dnx.chimera_graph(16)
pegasus_graph = dnx.pegasus_graph(16)

def get_count_embedding_physical_variables(embedding):
    result = 0
    for (key, ancillas) in embedding.items():
        result += len(ancillas)
    return result
def get_chain_lengths(embedding):
    result = []
    for (key, ancillas) in embedding.items():
        result.append(len(ancillas))
    return result

import time
import random
def get_statistics_embedding(target_graph, source_graph, num_runs, seed=1):
    random.seed(seed)
    times = []
    distribution_chain_lengths = []
    vars_in_qpu = []
    for _ in range(num_runs):
        embedding_seed = random.randint(0, 10000)
        t0 = time.perf_counter()
        embedding =  minorminer.find_embedding(source_graph, target_graph, random_seed=embedding_seed)
        t1 = time.perf_counter()
        times.append(round(t1-t0, 2))
        vars_in_qpu.append(get_count_embedding_physical_variables(embedding))
        distribution_chain_lengths.extend(get_chain_lengths(embedding))
    return times, distribution_chain_lengths, vars_in_qpu

import statistics
import pandas as pd
COL_FILE_NAME = 'file_name'
COL_WORDSIZE = 'wordsize'
COL_LOGIC_VARS = 'logic_vars'
COL_TOPOLOGY = 'topology'

COL_EMB_MAX_TIME = 'emb_max_time'
COL_EMB_MIN_TIME = 'emb_min_time'
COL_EMB_MEDIAN_TIME = 'emb_median_time'
COL_EMB_AVG_TIME = 'emb_avg_time'

COL_EMB_MAX_CHAIN = 'emb_max_chain'
COL_EMB_MIN_CHAIN = 'emb_min_chain'
COL_EMB_MEDIAN_CHAIN = 'emb_median_chain'
COL_EMB_AVG_CHAIN = 'emb_avg_chain'

COL_EMB_MAX_VARS = 'emb_max_vars'
COL_EMB_MIN_VARS = 'emb_min_vars'
COL_EMB_MEDIAN_VARS = 'emb_median_vars'
COL_EMB_AVG_VARS = 'emb_avg_vars'

columns = [COL_FILE_NAME, COL_WORDSIZE, COL_LOGIC_VARS, COL_EMB_MAX_TIME, COL_EMB_MIN_TIME, 
           COL_EMB_MEDIAN_TIME, COL_EMB_AVG_TIME, COL_EMB_MAX_CHAIN, 
           COL_EMB_MIN_CHAIN, COL_EMB_MEDIAN_CHAIN, COL_EMB_AVG_CHAIN,
          COL_EMB_MAX_VARS, COL_EMB_MIN_VARS, COL_EMB_MEDIAN_VARS, COL_EMB_AVG_VARS]

data_embedding = pd.DataFrame(columns=columns)
def add_embedding_statistics_row(filename, wordsize, topology, logic_vars, times, chains, vars_qpu):
    times.sort()
    chains.sort()
    vars_qpu.sort()
    
    row = {
        COL_FILE_NAME: filename,
        COL_WORDSIZE: wordsize,
        COL_LOGIC_VARS: logic_vars,
        COL_TOPOLOGY: topology,
        
        COL_EMB_MAX_TIME: max(times),
        COL_EMB_MIN_TIME: min(times),
        COL_EMB_MEDIAN_TIME: round(statistics.median(times),2),
        COL_EMB_AVG_TIME: round(sum(times)/len(times),2),
        
        COL_EMB_MAX_CHAIN: max(chains),
        COL_EMB_MIN_CHAIN: min(chains),
        COL_EMB_MEDIAN_CHAIN: round(statistics.median(chains),2),
        COL_EMB_AVG_CHAIN: round(sum(chains)/len(chains),2),
        
        COL_EMB_MAX_VARS: max(vars_qpu),
        COL_EMB_MIN_VARS: min(vars_qpu),
        COL_EMB_MEDIAN_VARS: round(statistics.median(vars_qpu),2),
        COL_EMB_AVG_VARS: round(sum(vars_qpu)/len(vars_qpu),2),
    }
    return data_embedding.append(row, ignore_index=True)
    

## Pegasus 

In [9]:
pegasus_times_32, pegasus_chain_length_32, pegasus_physical_vars_32 = get_statistics_embedding(pegasus_graph, problem_graph_32, 1)
data_embedding = add_embedding_statistics_row('d.btor2', 32, 'pegasus', num_variables_32, pegasus_times_32, pegasus_chain_length_32, pegasus_physical_vars_32)



In [11]:
data_embedding

,file_name,wordsize,logic_vars,emb_max_time,emb_min_time,emb_median_time,emb_avg_time,emb_max_chain,emb_min_chain,emb_median_chain,emb_avg_chain,emb_max_vars,emb_min_vars,emb_median_vars,emb_avg_vars,topology
0,d.btor2,32,1414,411.4,411.4,411.4,411.4,62,1,2.0,3.18,4498,4498,4498,4498.0,pegasus


In [12]:
data_embedding.to_csv('d_embedding.csv')

# Simulated Annealing

In [15]:
import neal
sampler = neal.SimulatedAnnealingSampler()
simulated_samples_32 = sampler.sample(bqm_32,num_reads=10000, seed=7).lowest()
print(len(simulated_samples_32.lowest()), simulated_samples_32.first.energy)

2 18.0


# Quantum Annealing

In [16]:
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite
ADVANTAGE = "Advantage_system4.1"

In [18]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=1.5)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

341.48 102.0 1


In [20]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=2)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

1021.91 126.0 2
